# **Analysis Hotel Booking Demand**

In [1]:
from custom_functions import *
import xgboost

In [2]:
#pip3 install xgboost

In [3]:
df_original = pd.read_csv('hotel_bookings.csv')

# Exploratory Data Analysis (EDA)

In [4]:
value_counts_csv(df_original)
count_nulls(df_original)
hist_boxplot(df_original)

In [5]:
# Number of rows and columns
df_original.shape

(119390, 32)

In [6]:
def min_max(column):
    upper = column.max()
    lower = column.min()
    y = (column - lower)/(upper-lower)
    return y

## - First Remarks and ideas for each column

- <span style='color:Orange'> **hotel** </span>: 2 classes, 66% City Hotel, 33% Resort Hotel
- <span style='color:Orange'> **is_canceled** </span> (target_label): 2 values, 0 (62%) or 1 (38%). It is quite unbalanced
- <span style='color:Orange'> **lead_time** </span>: many values, grouping by new categories? Skewed distributions towards the 0 value
- <span style='color:Orange'> **arrival_date_year** </span>: 3 years, 2015, 2016 and 2017
- <span style='color:Orange'> **arrival_date_month** </span>: many arrivals in June and July, it makes sense
- <span style='color:Orange'> **arrival_date_week_number** </span>: probably we can combine it with arrival_date_month. It has a quite normal distribution
- <span style='color:Orange'> **arrival_date_day_of_month** </span>: considering we have week_number and month_number we can eliminate this label
- <span style='color:Orange'> **stays_in_weekend_nights** </span>: skewed distribution towards 0, we could eliminate some big values
- <span style='color:Orange'> **stays_in_week_nights** </span>: same thought of previous label
- <span style='color:Orange'> **adults** </span>: 2 is 75% and 1 is 19%, there are some values with 0
- <span style='color:Orange'> **children** </span>: 93% has 0 children, we can think of eliminate this label or consider to create only a binary class, 4 <span style='color:Yellow'> **null** </span> values
- <span style='color:Orange'> **babies** </span>: 99% has 0 babies, same thought of children
- <span style='color:Orange'> **meal** </span>: 3 relevant classes, BB 77%, HB 12% and SC 8%
- <span style='color:Orange'> **country** </span>: 6 relevant classes, PRT 41%, GBR 10%, FRA 9%, ESP 7%, DEU 6%, ITA 3%, 0.4% of <span style='color:Yellow'> **null** </span> values
- <span style='color:Orange'> **market_segment** </span>: 5 relevant classes, Online TA 47%, Offline TA/TO 20%, Groups 17%, Direct 10% and Corporate 4.4%
- <span style='color:Orange'> **distribution_channel** </span>: 3 relevant classes TA/TO 82%, Direct 12% and Corporate 5%
- <span style='color:Orange'> **is_repeated_guest** </span>: the clients are quite new, 0 is 97% and 1 is 3%
- <span style='color:Orange'> **previous_cancellations** </span>: two relevant classes, 0 is about 95% and 1 is about 5%. We could transform this into a binary label
- <span style='color:Orange'> **previous_bookings_not_canceled** </span>: it is not relevant since we have previous cancellations.
- <span style='color:Orange'> **reserved_room_type** </span>: 4 relevant classes, A is 72%, D is 16%, E is 5%, F is 2.4 %
- <span style='color:Orange'> **assigned_room_type** </span>: 3 relevant classes, A is 62%, B 21% and E 6%. It could be interesting to know when the customer knows the **assigned_room_type**. If they knew this online some time before, it could be a reason for cancellation
- <span style='color:Orange'> **booking_changes** </span>: distribution is skewed towards 0, 3 relevant classes, 0 is 85%, 1 is 10% and 2 is 3%. More changes imply a high probable cancellation?
- <span style='color:Orange'> **deposit_type** </span>: only 3 classes, No deposit 87%, Non Refund 12% and Refundable 1%. We could Merge Refundable with No deposit? Non refund implies no cancellation?
- <span style='color:Orange'> **agent** </span>: too many values, it seems to be without important information, 13% <span style='color:Yellow'> **nulls** </span>. We can cancel this label
- <span style='color:Orange'> **company** </span>: 94% of rows are <span style='color:Yellow'> **null** </span>
- <span style='color:Orange'> **days_in_waiting_list** </span>: there are many values but 97% of rows is 0. Binary label?
- <span style='color:Orange'> **customer_type** </span>: only 4 classes, Transient is 75%, Transient-Party 21%, Contract is about 3.5% and Group is about 0.5%
- <span style='color:Orange'> **adr** </span>: there are some annoying outliers, we need to group the values somehow
- <span style='color:Orange'> **required_car_parking_spaces** </span>: only two relevant classes, 0 is about 94% and 1 is about 6%. Trasnform it in a binary label
- <span style='color:Orange'> **total_of_special_requests** </span>: only 3 relevant classes, 0 is 59%, 1 is 28% and 2 is 10%. Maybe we could group the information
- <span style='color:Orange'> **reservation_status** </span>: this is a posteriori information, we should eliminate it
- <span style='color:Orange'> **reservation_status_date** </span>: non-important information, eliminate this

# Feature Engineering I

### - Grouping, Treating outliers and null values

In [7]:
df = df_original.copy()

# lead_time column
df['lead_time'] = np.where(df['lead_time']>365, 365, df['lead_time'])

# arrival_date_month
df.drop('arrival_date_month', axis = 1, inplace = True)

# arrival_date_week_number
# Grouping considering first half of the month and second half
df['arrival_date_week_number'] = (df['arrival_date_week_number']-1) // 2

# arrival_date_day_of_month
df.drop('arrival_date_day_of_month', axis = 1, inplace = True)

# stays_in_weekend_nights
df['stays_in_weekend_nights'] = np.where(df['stays_in_weekend_nights']>2, 3, df['stays_in_weekend_nights'])

# stays_in_week_nights
df['stays_in_week_nights'] = np.where(df['stays_in_week_nights']>5, 6, df['stays_in_week_nights'])

# adults
df['adults'] = np.where(df['adults']>3, 3, df['adults'])
df['adults'].replace(0, df['adults'].median(), inplace = True)

# children
df['children'] = np.where(df['children']>0, 1, df['children'])
df['children'].replace(np.nan, 0, inplace = True)

# babies
df.drop('babies', axis = 1, inplace = True)

# meal
# We merge Undefined with SC (no service)
# and full breakfast with half breakfast
df['meal'].replace({'Undefined': 'SC', 'FB':'HB'}, inplace = True)

# country
countries = pd.read_csv('value_counts_csv/country.csv')
top_country = list(countries['country'].head(6))
df['country'] = np.where(df['country'].isin(top_country), df['country'], 'Others')

# market_segment and distribution_channel
# Further EDA for studying the relation between 
# market_segment and distribution_channel, they seem similar
df.groupby(['market_segment', 'distribution_channel']).size().unstack()

- <span style='color:Orange'> **Market segment** </span> seems to be a specialization of <span style='color:Orange'> **distribution channel** </span>. We decide to keep the former and drop the latter

In [20]:
df.drop('distribution_channel', axis = 1, inplace = True)

market_segments = pd.read_csv('value_counts_csv/market_segment.csv')
top_segments = list(market_segments['market_segment'].head(4))
df['market_segment'] = np.where(df['market_segment'].isin(top_segments), df['market_segment'], 'Others')

# previous_cancellations
df['previous_cancellations'] = np.where(df['previous_cancellations']>1, 1, 0)

# previous_bookings_not_canceled
df.drop('previous_bookings_not_canceled', axis = 1, inplace = True)

# reserved_room_type
room_types = pd.read_csv('value_counts_csv/reserved_room_type.csv')
top_room = list(room_types['reserved_room_type'].head(4))
df['reserved_room_type'] = np.where(df['reserved_room_type'].isin(top_segments), df['market_segment'], 'Others')

- We interpret the assigned_room_type label as assigned a posteriori. Therefore we decide to drop it

In [24]:
# assigned_room_type
df.drop('assigned_room_type', axis = 1, inplace = True)

# booking_changes
df['booking_changes'] = np.where(df['booking_changes']>2, 2, df['booking_changes'])

# deposit_type
df['deposit_type'].replace('Refundable', 'No Deposit', inplace = True)

# agent
df.drop('agent', axis = 1, inplace = True)

# company
df.drop('company', axis = 1, inplace = True)

# days_in_waiting_list
df['days_in_waiting_list'] = np.where(df['days_in_waiting_list']>0, 1, 0)

- We are confused about the interpretation of the <font color='orange'>**adr**</font> label and we decide not to consider it. We are not sure if it includes some a posteriori information. We can come back to it in further phase if some new interpreation arises.

In [30]:
# required_car_parking_spaces
df['required_car_parking_spaces'] = np.where(df['required_car_parking_spaces']>0, 1, 0)

# total_of_special_requests
df['total_of_special_requests'] = np.where(df['total_of_special_requests']>1, 2, df['total_of_special_requests'])

# reservation_status and reservation_status_date
df.drop(['reservation_status', 'reservation_status_date'], axis = 1, inplace = True)

### - Normalization

# OLD CODE

In [ ]:
python setup.py install--user

In [ ]:
conda install -c conda-forge xgboost

In [ ]:
df_model = df.copy()
df_model = df_model.select_dtypes(exclude=['object'])
df_model = pd.get_dummies(df_model)

In [ ]:
df_model.shape
#df.shape

In [ ]:
X = df_model.drop('is_canceled', axis = 1)
y = df_model['is_canceled']

X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
y.value_counts()

In [ ]:
#Import libraries:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import precision_score, recall_score, accuracy_score, roc_auc_score

In [ ]:
D_train = xgb.DMatrix(X_train, label=Y_train)
D_test = xgb.DMatrix(X_test, label=Y_test)

param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'multi:softprob',  
    'num_class': 2} 

steps = 20  # The number of training iterations

model = xgb.train(param, D_train, steps)

preds = model.predict(D_test)
print(preds)

best_preds = np.asarray([np.argmax(line) for line in preds])
print(best_preds)

print("Precision = {}".format(precision_score(Y_test, best_preds, average='macro')))
print("Recall = {}".format(recall_score(Y_test, best_preds, average='macro')))
print("Accuracy = {}".format(accuracy_score(Y_test, best_preds)))

In [ ]:
xgb_clf = xgb.XGBClassifier()

parameters = {
     "eta"    : [0.20, 0.30] ,
     "max_depth"        : [ 3, 6],
     "gamma"            : [ 0.1, 0.2]
     }

grid = GridSearchCV(xgb_clf,
                    parameters,
                    scoring="neg_log_loss",
                    cv=3)

model = grid.fit(X_train, Y_train)
#predictions = model.predict(X_test)

#model.dump_model('dump.raw.txt')

In [ ]:
preds = model.predict(X_test)
print(preds)

In [ ]:
model.best_estimator_

In [ ]:
model.best_params_
model.best_estimator_
cvres = model.cv_results_

for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [ ]:
xgb_model = xgb.XGBClassifier()

#brute force scan for all parameters, here are the tricks
#usually max_depth is 6,7,8
#learning rate is around 0.05, but small changes may make big diff
#tuning min_child_weight subsample colsample_bytree can have 
#much fun of fighting against overfit 
#n_estimators is how many round of boosting
#finally, ensemble xgboost with multiple seeds may reduce variance
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['binary:logistic'],
              'learning_rate': [0.05], #so called `eta` value
              'max_depth': [6],
              'min_child_weight': [11],
              'silent': [1],
              'subsample': [0.8],
              'colsample_bytree': [0.7],
              'n_estimators': [5], #number of trees, change it to 1000 for better results
              'missing':[-999],
              'seed': [1337]}


clf = GridSearchCV(xgb_model, parameters, n_jobs=5, 
                   cv=StratifiedKFold(train['QuoteConversion_Flag'], n_folds=5, shuffle=True), 
                   scoring='roc_auc',
                   verbose=2, refit=True)

clf.fit(train[features], train["QuoteConversion_Flag"])

#trust your CV!
best_parameters, score, _ = max(clf.grid_scores_, key=lambda x: x[1])
print('Raw AUC score:', score)
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

test_probs = clf.predict_proba(test[features])[:,1]

sample = pd.read_csv('../input/sample_submission.csv')
sample.QuoteConversion_Flag = test_probs
sample.to_csv("xgboost_best_parameter_submission.csv", index=False)

In [ ]:
def classifier_gridCV(X_train, y_train, clf, 
                      X_test = None, y_test = None, cv = 3, scoring = 'accuracy', params = {}, model_name = "model"):
    
    grid = GridSearchCV(clf, params, cv = cv, scoring = scoring, refit = True)
    
    model = grid.fit(X_train, y_train)
    print("The best parameters of grid are: ", model.best_params_, 
          "\nThe best estimator is: ", model.best_estimator_)
    
    if not os.path.exists('Models/CV_results'):
        os.makedirs('Models/CV_results')
    
    cvres = model.cv_results_
    
    if params != {}:    
        dataframe = pd.DataFrame(cvres["params"])
        dataframe.insert(0, "mean_test_score", cvres["mean_test_score"])
                        
    else: 
        dataframe = pd.DataFrame({"mean_test_score":cvres["mean_test_score"]})
            
    dataframe.to_csv("./Models/CV_results/CV_results_"+model_name+".csv", index = False) 
    
    if X_test is not None:
        results = model.predict(X_test)
    
        if y_test is not None:
            print("Precision = {}".format(precision_score(y_test, results, average='macro')))
            print("Recall = {}".format(recall_score(y_test, results, average='macro')))
            print("Accuracy = {}".format(accuracy_score(y_test, results)))
            
    return(model)
    

In [ ]:
classifier_gridCV(X_train, Y_train, xgb.XGBClassifier(), X_test, Y_test, 
                  model_name = "xg_boost_2")